In [1]:
import os

os.chdir('../')

In [2]:
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout
import pandas as pd

data_train_path = "Dataset/data_train.csv"

URM_all_dataframe = pd.read_csv(data_train_path)
ICM_all_dataframe = pd.read_csv('Dataset/data_ICM_metadata.csv')

ICM_all = sps.coo_matrix((ICM_all_dataframe['data'], (ICM_all_dataframe['item_id'], ICM_all_dataframe['feature_id'])))
ICM_all = ICM_all.tocsr()

URM_all_dataframe = pd.read_csv('Dataset/data_train.csv')
URM_all = sps.coo_matrix((URM_all_dataframe['data'], (URM_all_dataframe['user_id'], URM_all_dataframe['item_id'])))
URM_all = URM_all.tocsr()

#ICM_train_validation, ICM_test = split_train_in_two_percentage_global_sample(ICM_all, train_percentage=0.8)
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage=0.8)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions


In [3]:
from Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
import optuna

recommender = ItemKNNCBFRecommender(URM_train_validation, ICM_all)

def objective_function(trial):
    topK = trial.suggest_int('topK', 10, 500)  # Reduced range
    shrink = trial.suggest_int('shrink', 0, 700)  # Reduced range
    similarity = trial.suggest_categorical('similarity', ["jaccard", "tversky", "dice", "cosine"])  # Focused similarity
    feature_weighting = trial.suggest_categorical('feature_weighting', ["BM25", "TF-IDF", "none"])  # Focused feature_weighting

    recommender.fit(topK=topK, shrink=shrink, similarity=similarity, feature_weighting=feature_weighting)

    result_dict, _ = evaluator_test.evaluateRecommender(recommender)

    return result_dict.loc[10]["MAP"]

class SaveResults(object):
    def __init__(self):
        self.results_df = pd.DataFrame(columns=["result"])
    
    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]
        self.results_df = pd.concat([self.results_df, pd.DataFrame([hyperparam_dict])], ignore_index=True)
        self.results_df.to_csv("logs/KNN/KNN_CBF_firstTrial.csv", index=False)

optuna_study = optuna.create_study(study_name="KNN_CBF", direction="maximize")

save_results = SaveResults()

optuna_study.optimize(
    objective_function,
    callbacks=[save_results],
    n_trials=100
)


[I 2025-01-05 22:47:05,230] A new study created in memory with name: KNN_CBF


Similarity column 38121 (100.0%), 1117.02 column/sec. Elapsed time 34.13 sec
EvaluatorHoldout: Processed 35595 (100.0%) in 20.08 sec. Users per second: 1773


[I 2025-01-05 22:47:59,725] Trial 0 finished with value: 0.01649194531958995 and parameters: {'topK': 342, 'shrink': 509, 'similarity': 'cosine', 'feature_weighting': 'BM25'}. Best is trial 0 with value: 0.01649194531958995.


Similarity column 38121 (100.0%), 1116.61 column/sec. Elapsed time 34.14 sec
EvaluatorHoldout: Processed 35595 (100.0%) in 19.58 sec. Users per second: 1818


[I 2025-01-05 22:48:53,620] Trial 1 finished with value: 0.01660464284042058 and parameters: {'topK': 313, 'shrink': 537, 'similarity': 'cosine', 'feature_weighting': 'none'}. Best is trial 1 with value: 0.01660464284042058.


...

Similarity column 38121 (100.0%), 1129.41 column/sec. Elapsed time 33.75 sec
EvaluatorHoldout: Processed 35595 (100.0%) in 12.15 sec. Users per second: 2930


[I 2025-01-05 22:50:33,357] Trial 3 finished with value: 0.01442992260817847 and parameters: {'topK': 38, 'shrink': 59, 'similarity': 'dice', 'feature_weighting': 'BM25'}. Best is trial 1 with value: 0.01660464284042058.


Similarity column 38121 (100.0%), 1117.97 column/sec. Elapsed time 34.10 sec
EvaluatorHoldout: Processed 35595 (100.0%) in 20.76 sec. Users per second: 1715


[I 2025-01-05 22:51:28,676] Trial 4 finished with value: 0.011327487140382548 and parameters: {'topK': 381, 'shrink': 207, 'similarity': 'jaccard', 'feature_weighting': 'TF-IDF'}. Best is trial 1 with value: 0.01660464284042058.
[W 2025-01-05 22:51:28,684] Trial 5 failed with parameters: {'topK': 144, 'shrink': 306, 'similarity': 'dice', 'feature_weighting': 'TF-IDF'} because of the following error: AssertionError('TF_IDF: Data matrix contains 38121 negative values, computing the square root is not possible.').
Traceback (most recent call last):
  File "/Users/lucapagano/miniconda3/envs/RecSysPD/lib/python3.8/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_56802/987174676.py", line 12, in objective_function
    recommender.fit(topK=topK, shrink=shrink, similarity=similarity, feature_weighting=feature_weighting)
  File "/Users/lucapagano/Developer/RecSys/Competizione/Re

AssertionError: TF_IDF: Data matrix contains 38121 negative values, computing the square root is not possible.